<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65


    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-19 09:29:06
-------------------
qualified stocks: 94
with latest results: 28
still star stocks: 18
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  1.01 L
Current:  1.36 C
-------------------
Today PnL: 14.81 K (0.11%)
Current PnL: -18.48 L (-12.57%)
CY Booked + Current PnL: -10.21 L (-6.94%)
-------------------
Total profit:  3.55 L
Total loss:  -22.02 L
-------------------
Total Booked + Current PnL: 16.77 L (13.97%)
Total Booked PnL: 35.25 L (29.36%)
Curr Year Booked PnL: 8.27 L (6.06%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.14 C
Est FTT PnL: 78.10 L (57.26%)
Deployed:  1.20 C
Current:  1.36 C
CAGR/XIRR %: 8.17%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,HINDALCO,651.95,756.01,-18.25,H-LC,6.80,113320.0,9008.0,7638.0,-0.83,8.64,6.74,15.96,9.0,1.18,0.84,26.99,X5K,ATH,METALS
52,LALPATHLAB,2838.53,3545.00,-21.93,H-SC,11.67,233056.0,40036.0,7994.0,-0.04,20.74,3.43,24.89,137.0,5.01,1.72,48.93,X40N,NTT,HEALTHCARE
69,SBIN,760.30,863.00,-11.34,M-LC,3.88,219552.0,18072.0,9133.0,0.10,8.97,4.16,13.51,60.0,1.98,1.62,22.70,XY25,NTT,BANKS
4,APOLLOHOSP,7098.95,8285.00,-13.64,H-LC,6.60,172491.0,16314.0,9780.0,0.36,10.45,5.67,16.71,22.0,1.67,1.27,29.71,X40N,BTT,HEALTHCARE
56,MEDANTA,1087.93,1486.00,10.70,H-MC,13.91,158108.0,31908.0,14261.0,1.47,25.28,9.02,36.59,108.0,2.24,1.17,40.97,XY24,NTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,RAJOOENG,102.50,143.33,-32.22,H-SC,7.60,102710.0,210.0,40622.0,1.06,0.20,39.55,39.83,114.0,0.01,0.76,1.69,AR,ATH,MISC
59,PGHH,13388.00,18062.58,-30.81,H-MC,6.09,200250.0,-570.0,70688.0,-0.21,-0.28,35.30,34.92,80.0,-0.01,1.48,4.38,X40,ATH,FMCG
57,NATIONALUM,189.63,247.44,-49.46,H-MC,1.98,100032.0,-472.0,31110.0,0.22,-0.47,31.10,30.49,79.0,-0.02,0.74,31.78,MH,ATH,METALS


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,ITC,411.90,452.0,-38.77,H-LC,2.98,147675.0,-2257.0,16850.0,-0.15,-1.51,11.41,9.74,4.0,-0.13,1.09,4.64,X40,NTT,FMCG
21,CIPLA,1492.70,1795.0,-19.78,H-LC,3.10,213638.0,9138.0,32281.0,-0.37,4.47,15.11,20.25,10.0,0.28,1.58,13.59,X40N,BTT,PHARMA
58,NESTLEIND,1134.30,1377.0,-13.82,H-LC,3.67,265239.0,-187.0,56973.0,-0.93,-0.07,21.48,21.40,11.0,-0.00,1.96,7.25,XY25,NTT,FMCG
54,LTIM,5564.16,7230.2,-7.80,H-LC,3.74,182916.0,-17394.0,77373.0,-0.17,-8.68,42.30,29.94,16.0,-0.22,1.35,26.37,X200,ATH,IT
5,ASIANPAINT,2961.56,4250.0,-9.79,H-LC,4.22,175943.0,-25443.0,113061.0,-0.00,-12.63,64.26,43.51,27.0,-0.23,1.30,22.47,X40,BTT,PAINTS


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,TTKPRESTIG,769.29,770.00,100.85,M-SC,9.86,86539.0,-14238.0,14331.0,2.57,-14.13,16.56,0.09,245.0,-0.99,0.64,13.69,OX40N,NTT,DURABLES
61,RAJESHEXPO,517.65,518.00,1668.94,L-SC,7.14,48249.0,-88928.0,89024.0,2.10,-64.83,184.51,0.07,267.0,-1.00,0.36,20.00,OX40N,NTT,JEWELLERY
83,TRIDENT,37.35,48.00,-9.12,M-SC,3.68,70790.0,-21464.0,47769.0,1.99,-23.27,67.48,28.51,224.0,-0.45,0.52,21.13,XR,NTT,TEXTILES
51,KPIGREEN,497.21,731.35,10.94,H-SC,11.38,131985.0,6688.0,52319.0,1.68,5.34,39.64,47.09,141.0,0.13,0.97,65.19,MH,ATH,POWER
90,VBL,492.64,671.64,-9.95,H-LC,9.76,273250.0,8210.0,88096.0,1.62,3.10,32.24,36.33,5.0,0.09,2.02,16.81,X40N,ATH,FMCG


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,SYMPHONY,1306.42,1306.0,-40.95,M-SC,13.86,125943.0,-45198.0,45138.0,-1.66,-26.41,35.84,-0.03,196.0,-1.00,0.93,1.81,OX40N,NTT,DURABLES
18,CAMPUS,294.86,393.0,-28.59,M-SC,2.92,148737.0,-19038.0,74874.0,-1.13,-11.35,50.34,33.28,210.0,-0.25,1.10,15.15,XY24,NTT,FOOTWEAR
92,WHIRLPOOL,1219.98,2270.0,-42.94,M-SC,4.77,96495.0,4996.0,73751.0,-1.02,5.46,76.43,86.07,223.0,0.07,0.71,39.59,XR,NTT,DURABLES
13,BATAINDIA,1550.24,2096.0,-38.44,M-SC,7.76,92047.0,-36623.0,81922.0,-1.01,-28.46,89.00,35.20,219.0,-0.45,0.68,5.65,X40,NTT,FOOTWEAR
10,BAJAJHFL,122.26,152.0,-11.52,H-MC,5.93,186435.0,-14071.0,62847.0,-0.95,-7.02,33.71,24.33,90.0,-0.22,1.38,7.51,X40N,BTT,FINANCE


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,TTKPRESTIG,769.29,770.0,100.85,M-SC,9.86,86539.0,-14238.0,14331.0,2.57,-14.13,16.56,0.09,245.0,-0.99,0.64,13.69,OX40N,NTT,DURABLES
20,CERA,7989.07,9475.0,-16.95,H-SC,3.04,118584.0,-25219.0,51964.0,1.37,-17.54,43.82,18.60,149.0,-0.49,0.88,30.86,OX40N,NTT,CERAMICS
43,INDIGOPNTS,1407.73,1408.0,117.63,M-SC,2.28,142774.0,-31785.0,31824.0,0.99,-18.21,22.29,0.02,221.0,-1.00,1.05,23.81,OX40N,NTT,PAINTS
85,UJJIVANSFB,52.77,60.0,83.20,H-SC,16.87,115857.0,-26622.0,46146.0,-0.46,-18.68,39.83,13.70,163.0,-0.58,0.86,36.74,OX40N,NTT,BANKS
50,KANSAINER,299.63,340.0,-68.70,H-SC,3.22,215172.0,-54495.0,90824.0,0.94,-20.21,42.21,13.47,138.0,-0.60,1.59,8.83,XY24,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,TCS,3794.03,4998.00,-29.01,H-LC,16.19,235287.0,-60647.0,154560.0,0.18,-20.49,65.69,31.73,1.0,-0.39,1.74,0.45,X40,BTT,IT
45,INFY,1461.46,2275.00,-21.03,H-LC,11.59,258011.0,-3590.0,149208.0,0.39,-1.37,57.83,55.67,3.0,-0.02,1.91,4.65,X40,BTT,IT
47,ITC,411.90,452.00,-38.77,H-LC,2.98,147675.0,-2257.0,16850.0,-0.15,-1.51,11.41,9.74,4.0,-0.13,1.09,4.64,X40,NTT,FMCG
90,VBL,492.64,671.64,-9.95,H-LC,9.76,273250.0,8210.0,88096.0,1.62,3.10,32.24,36.33,5.0,0.09,2.02,16.81,X40N,ATH,FMCG
1,ABB,5551.76,7934.00,-42.30,H-LC,11.30,196072.0,-20447.0,113349.0,-0.20,-9.44,57.81,42.91,7.0,-0.18,1.45,2.68,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,LALPATHLAB,2838.53,3545.0,-21.93,H-SC,11.67,233056.0,40036.0,7994.0,-0.04,20.74,3.43,24.89,137.0,5.01,1.72,48.93,X40N,NTT,HEALTHCARE
56,MEDANTA,1087.93,1486.0,10.70,H-MC,13.91,158108.0,31908.0,14261.0,1.47,25.28,9.02,36.59,108.0,2.24,1.17,40.97,XY24,NTT,HEALTHCARE
28,DMART,3764.92,5201.0,-9.15,H-LC,7.17,226816.0,42335.0,28034.0,1.56,22.95,12.36,38.14,38.0,1.51,1.68,35.99,X40N,NTT,FMCG


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,BLUESTARCO,1646.70,2326.38,11.32,H-MC,6.24,192180.0,27510.0,40454.0,0.32,16.71,21.05,41.28,89.0,0.68,1.42,26.02,X40N,ATH,AC
51,KPIGREEN,497.21,731.35,10.94,H-SC,11.38,131985.0,6688.0,52319.0,1.68,5.34,39.64,47.09,141.0,0.13,0.97,65.19,MH,ATH,POWER
56,MEDANTA,1087.93,1486.00,10.70,H-MC,13.91,158108.0,31908.0,14261.0,1.47,25.28,9.02,36.59,108.0,2.24,1.17,40.97,XY24,NTT,HEALTHCARE
91,VOLTAS,1278.28,1918.49,-3.18,H-MC,7.66,203145.0,11403.0,84630.0,0.38,5.95,41.66,50.08,99.0,0.13,1.50,13.15,XY25,ATH,AC
19,CAMS,3643.00,5211.76,-4.50,H-SC,4.58,108696.0,6692.0,37228.0,-0.38,6.56,34.25,43.06,122.0,0.18,0.80,24.80,X40N,ATH,MISC


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-4.90,H-MC,9.35,175980.0,25067.0,32240.0,0.88,16.61,18.32,37.97,93.0,0.78,1.30,46.12,AR,ATH,AUTO
51,KPIGREEN,497.21,731.35,10.94,H-SC,11.38,131985.0,6688.0,52319.0,1.68,5.34,39.64,47.09,141.0,0.13,0.97,65.19,MH,ATH,POWER
42,INDIAMART,2327.09,4850.92,-50.53,H-SC,11.89,137917.0,14581.0,119188.0,1.14,11.82,86.42,108.45,119.0,0.12,1.02,36.82,AR,ATH,MISC
92,WHIRLPOOL,1219.98,2270.00,-42.94,M-SC,4.77,96495.0,4996.0,73751.0,-1.02,5.46,76.43,86.07,223.0,0.07,0.71,39.59,XR,NTT,DURABLES
93,WIPRO,243.46,420.00,-13.34,M-LC,7.46,152576.0,1631.0,107825.0,0.36,1.08,70.67,72.51,53.0,0.02,1.13,7.17,XR,NTT,IT


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-49.70,L-SC,21.40,74170.0,-39379.0,79429.0,1.47,-34.68,107.09,35.27,268.0,-0.50,0.55,81.53,XR,NTT,HOTELS
3,ALKYLAMINE,2347.98,4546.37,-2.82,H-SC,15.48,90382.0,-10581.0,105114.0,0.77,-10.48,116.30,93.63,148.0,-0.10,0.67,34.62,SR,ATH,CHEMICALS
55,MASFIN,326.60,399.50,-18.70,H-SC,12.65,92535.0,-5445.0,27316.0,0.54,-5.56,29.52,22.32,152.0,-0.20,0.68,34.21,XR,ATH,FINANCE
89,VALIANTORG,512.64,838.00,-326.45,H-SC,16.04,96200.0,-37086.0,121683.0,-0.05,-27.82,126.49,63.47,139.0,-0.30,0.71,61.26,XR,NTT,CHEMICALS
9,BAJAJFINSV,1994.80,2758.00,-9.59,H-LC,7.12,97059.0,-686.0,38086.0,-0.64,-0.70,39.24,38.26,35.0,-0.02,0.72,27.77,X40,BTT,FINANCE


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-4.90,H-MC,9.35,175980.0,25067.0,32240.0,0.88,16.61,18.32,37.97,93.0,0.78,1.30,46.12,AR,ATH,AUTO
52,LALPATHLAB,2838.53,3545.00,-21.93,H-SC,11.67,233056.0,40036.0,7994.0,-0.04,20.74,3.43,24.89,137.0,5.01,1.72,48.93,X40N,NTT,HEALTHCARE
28,DMART,3764.92,5201.00,-9.15,H-LC,7.17,226816.0,42335.0,28034.0,1.56,22.95,12.36,38.14,38.0,1.51,1.68,35.99,X40N,NTT,FMCG
5,ASIANPAINT,2961.56,4250.00,-9.79,H-LC,4.22,175943.0,-25443.0,113061.0,-0.00,-12.63,64.26,43.51,27.0,-0.23,1.30,22.47,X40,BTT,PAINTS
4,APOLLOHOSP,7098.95,8285.00,-13.64,H-LC,6.60,172491.0,16314.0,9780.0,0.36,10.45,5.67,16.71,22.0,1.67,1.27,29.71,X40N,BTT,HEALTHCARE


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.45
1,25,40.94
2,50,70.51


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    38.55
LC    35.50
MC    25.99
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.09
X40N     15.42
X40      14.15
XY25     11.51
XR        8.85
AR        8.26
OX40N     7.38
X5K       2.22
MH        1.71
X200      1.35
SR        1.10
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    27.37
H-SC    24.05
H-MC    23.10
M-SC    13.04
M-LC     7.07
M-MC     2.56
L-SC     1.46
L-LC     1.06
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.10,-1.56,32.66
IT,11.60,-18.93,78.33
FINANCE,8.04,-19.89,64.24
BANKS,7.41,-13.57,61.79
MISC,5.72,-18.54,86.50
PAINTS,5.58,-15.51,38.28
HEALTHCARE,5.20,6.85,16.66
ELECTRICAL,4.65,-11.61,59.94
AUTO,4.46,-11.86,60.52


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2833657.0,23
XR,1094692.0,13
AR,1092021.0,10
X40,822273.0,11
X40N,520519.0,12
XY25,491652.0,8
OX40N,460435.0,10
SR,194748.0,2
X5K,139012.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2762453.0,28
M-SC,1476958.0,17
H-MC,1252250.0,18
H-LC,1199610.0,19
M-LC,421603.0,5
M-MC,331557.0,2
L-SC,264176.0,3
L-MC,57966.0,1
L-LC,43238.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,936062.0,6
M-SC,XY24,867767.0,7
H-SC,AR,664712.0,5
H-LC,X40,528615.0,6
H-SC,XR,516384.0,6
H-MC,XY24,485837.0,5
M-MC,XY24,331557.0,2
H-LC,AR,234434.0,2
H-MC,X40,211736.0,4


In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 43.0 seconds
